## SELECCIÓN DE CARACTERÍSTICAS A TRAVÉS DE ALGORÍTMO GENÉTICO

In [1]:
import numpy as np
import random
import warnings
import copy
import pandas as pd
import time
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import rankdata
from sklearn import datasets, metrics
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score, ShuffleSplit

In [2]:
class Individuo:

    def __init__(self, n_variables, n_max=None, n_min=1, n_max_estricto=False,
                 verbose=False):

        # Número de variables del individuo
        self.n_variables = n_variables
        # Número máximo de predictores incluidos
        self.n_max = n_max
        self.n_max_estricto = n_max_estricto
        # Número mínimo de predictores incluidos
        self.n_min = n_min
        # Secuencia del individuo
        self.secuencia = None
        # Índices de las columans empleadas como predictores
        self.predictores = None
        # Número predictores incluidos
        self.n_predictores_incluidos = None
        # Fitness del individuo
        self.fitness = None
        # Métrica
        self.metrica = None
        # Valor de la métrica con la que se calcula el fitness
        self.valor_metrica = None
        # Modelo empleado para evaluar el individuo
        self.modelo = None
        # Tipo de modelo: regresion o clasificación
        self.tipo_modelo = None

        # COMPROBACIONES INICIALES: EXCEPTIONS Y WARNINGS
        # ----------------------------------------------------------------------
        if self.n_max is not None and self.n_max > self.n_variables:
            raise Exception(
                "El valor de n_max no puede ser superior al de n_variables"
            )

        # COMPROBACIONES INICIALES: ACCIONES
        # ----------------------------------------------------------------------
        # Si no se especifica n_max, se emplea por defecto el valor de 
        # n_variables.
        if self.n_max is None:
            self.n_max = n_variables

        # CREACIÓN DE LA SECUENCIA BOLEANA QUE DEFINE AL INDIVIDUO
        # ----------------------------------------------------------------------
        # Se crea un array boleano que representa el individuo.
        self.secuencia =  np.full(
                            shape      = self.n_variables,
                            fill_value = False,
                            dtype      = "bool"
                          )
        # Se selecciona (con igual probabilidad) el número de valores TRUE 
        # que puede tener el individuo, dentro del rango acotado por n_min y
        # n_max.
        n_true = np.random.randint(
                    low  = self.n_min,
                    high = self.n_max + 1,
                    size = None)

        # Se sustituyen n_true posiciones aleatorias por valores True.
        posiciones_true = np.random.choice(
                            a       = self.n_variables,
                            size    = n_true,
                            replace = False
                          )
        self.secuencia[posiciones_true] = True
        self.n_predictores_incluidos    = sum(self.secuencia)

        # Se identifican los indices de las posiciones True
        self.predictores = np.arange(self.n_variables)[self.secuencia]

        # INFORMACIÓN DEL PROCESO (VERBOSE)
        # ----------------------------------------------------------------------
        if verbose:
            print("----------------------")
            print("Nuevo individuo creado")
            print("----------------------")
            print("Secuencia: " + str(self.secuencia))
            print("Índice predictores: " + str(self.predictores))
            print("Número de predictores incluidos: " \
                  + str(self.n_predictores_incluidos))
            print("Fitness: " + str(self.fitness))
            print("Métrica: "+ str(self.metrica))
            print("Valor métrica: "+ str(self.valor_metrica))
            print("Modelo empleado para calcular fitness: " \
                  + str(self.modelo))
            print("Tipo de modelo: " + str(self.tipo_modelo))
            print("")

    def __repr__(self):

        texto = "Individuo" \
                + "\n" \
                + "---------" \
                + "\n" \
                + "Secuencia: " + str(self.secuencia) \
                + "\n" \
                + "Índice predictores: " + str(self.predictores) \
                + "\n" \
                + "Número de predictores incluidos: " \
                + str(self.n_predictores_incluidos) \
                + "\n" \
                + "Fitness: " + str(self.fitness) \
                + "\n" \
                + "Métrica: " + str(self.metrica) \
                + "\n" \
                + "Valor métrica: " + str(self.valor_metrica) \
                + "\n" \
                + "Modelo empleado para calcular fitness: " + str(self.modelo) \
                + "\n" \
                + "Tipo de modelo: " + str(self.tipo_modelo) \
                + "\n"

        return(texto)

    def evaluar_individuo(self, x, y, tipo_modelo, modelo, metrica, cv=5,
                          test_size=0.2, cv_seed=123, nivel_referencia = None,
                          rf_n_estimators = 100, verbose = False):

        # COMPROBACIONES INICIALES: EXCEPTIONS Y WARNINGS
        # ----------------------------------------------------------------------
        if metrica not in ["neg_mean_squared_error",
                           "neg_mean_absolute_error",
                           "f1", "accuracy"]:
            raise Exception(
                "La métrica debe ser: neg_mean_squared_error, " \
                 + "neg_mean_absolute_error, f1 o accuracy"
            )
        
        if self.n_variables != x.shape[1]:
            raise Exception(
                "n_variables debe ser igual al número de columnas de x."
            )

        if not isinstance(x, np.ndarray) or x.ndim != 2:
            raise Exception(
                "x debe ser un array numpy de dos dimensiones (una matriz)."
            )
        if not isinstance(y, np.ndarray) or y.ndim != 1:
            raise Exception(
                "y debe ser un array numpy de 1 dimensiones."
            )

        if modelo == "lineal":
            if metrica not in ["neg_mean_squared_error", "neg_mean_absolute_error"]:
                raise Exception(
                "Para el modelo lineal, la metrica debe ser: " \
                + "neg_mean_squared_error, neg_mean_absolute_error."
                )
            if tipo_modelo != "regresion":
                raise Exception(
                "El modelo lineal solo puede aplicarse a problemas de regresión."
                )

        if modelo == "glm":
            if metrica not in ["f1", "accuracy"]:
                raise Exception(
                "Para el modelo glm la métrica de evaluación debe ser f1," \
                + "o accuracy."
                )
            if tipo_modelo != "clasificacion":
                raise Exception(
                "El modelo glm solo puede aplicarse a problemas de clasificación."
                )
            if len(np.unique(y)) != 1:
                raise Exception(
                "El modelo glm solo puede aplicarse a problemas de clasificación" \
                + "binaria."
                )

        # Se identifica el modelo y el tipo
        self.modelo = modelo
        self.tipo_modelo = tipo_modelo
        self.metrica = metrica

        # Se selecciona la clase de scikit-learn correspondiente al modelo
        if self.modelo == "lineal":
            mod = LinearRegression()
        elif self.modelo == "glm":
            mod = LogisticRegression()
        elif self.modelo == "randomforest" and self.tipo_modelo == "regresion":
            mod = RandomForestRegressor(
                    n_estimators = rf_n_estimators,
                    random_state = 1234,
                    bootstrap    = False
                  )      
        elif self.modelo == "randomforest" and self.tipo_modelo == "clasificacion":
            mod = RandomForestClassifier(
                    n_estimators = rf_n_estimators,
                    random_state = 1234,
                    bootstrap    = False
                  ) 
        
        cv = ShuffleSplit(
                n_splits     = cv,
                test_size    = test_size,
                random_state = cv_seed
             )

        metrica_cv = cross_val_score(
                        estimator = mod,
                        X         = x[:,self.predictores],
                        y         = y,
                        cv        = cv,
                        scoring   = metrica,
                        n_jobs    = 1
                     )

        self.valor_metrica = metrica_cv.mean()
        self.fitness       = metrica_cv.mean()

        # INFORMACIÓN DEL PROCESO (VERBOSE)
        # ----------------------------------------------------------------------
        if verbose:
            print("El individuo ha sido evaluado")
            print("-----------------------------")
            print("Métrica: " + str(self.metrica))
            print("Valor métrica: " + str(self.valor_metrica))
            print("Fitness: " + str(self.fitness))
            print("")
    def mutar(self, prob_mut=0.01, verbose=False):


        # COMPROBACIONES INICIALES: EXCEPTIONS Y WARNINGS
        # ----------------------------------------------------------------------
        if prob_mut < 0 or prob_mut > 1:
            raise Exception(
                "El argumento prob_mut debe de estar en el rango [0,1]."
            )

        # SELECCIÓN PROBABILISTA DE POSICIONES (VARIABLES) QUE MUTAN
        #-----------------------------------------------------------------------
        posiciones_mutadas = np.random.uniform(
                                low=0,
                                high=1,
                                size=self.n_variables
                             )
        posiciones_mutadas = posiciones_mutadas < prob_mut

        # Se modifican las posiciones de la secuencia del individuo que coinciden 
        # con las posiciones_mutadas.
        self.secuencia[posiciones_mutadas] = \
                np.logical_not(self.secuencia[posiciones_mutadas])
        
        # Todo individuo debe tener como mínimo 1 predictor, si como consecuencia de la 
        # mutación, ningun valor de la secuencia es True, se selecciona una posición
        # aleatoria y se sobreescribe con True.
        
        if sum(self.secuencia == True) == 0:
            indice = np.random.choice(
                        a       = np.arange(self.n_variables),
                        size    = 1, 
                        replace = False
                      )
            self.secuencia[indice] = True

        # Se actualiza el indice de los predictores incluidos.
        self.predictores = np.arange(self.n_variables)[self.secuencia]
        # Se actualiza el número total de predictores incluidos.
        self.n_predictores_incluidos = sum(self.secuencia)
        
        # INFORMACIÓN DEL PROCESO (VERBOSE)
        # ----------------------------------------------------------------------
        if verbose:
            print("El individuo ha sido mutado")
            print("---------------------------")
            print("Total mutaciones: " + str(np.sum(posiciones_mutadas)))
            print("Secuencia: " + str(self.secuencia))
            print("Índice predictores: " + str(self.predictores))
            print("")  
            
    def forzar_n_max(self):

        #Se identifica si el número de True es la secuencia supera a n_max.
        n_exceso = sum(self.secuencia) - self.n_max

        if n_exceso > 0:
            # Se seleccionan aleatoriamente n_max posiciones con valor True en 
            # la secuencia del individuo.
            indices = np.random.choice(
                        a       = np.flatnonzero(self.secuencia == True),
                        size    = n_exceso, 
                        replace = False
                      )
            self.secuencia[indices] = \
                np.logical_not(self.secuencia[indices])

            # Se actualiza el indice de los predictores incluidos.
            self.predictores = np.arange(self.n_variables)[self.secuencia]
            # Se actualiza el número total de predictores incluidos.
            self.n_predictores_incluidos = sum(self.secuencia)

In [3]:
class Poblacion:
    
    def __init__(self, n_individuos, n_variables, n_max=None,
                 n_min=1, n_max_estricto=False, verbose=False):

        # Número de individuos de la población
        self.n_individuos = n_individuos
        # Número de variables de cada individuo
        self.n_variables = n_variables
        # Número máximo de predictores incluidos
        self.n_max = n_max
        self.n_max_estricto = n_max_estricto
        # Número mínimo de predictores incluidos
        self.n_min = n_min
        # Métrica utilizada en la evaluación de los individuos
        self.metrica = None
        # Modelo empleado para evaluar el individuo
        self.modelo = None
        # Lista de los individuos de la población
        self.individuos = []
        # Etiqueta para saber si la población ha sido evaluada
        self.evaluada = False
        # Etiqueta para saber si la población ha sido optimizada
        self.optimizada = False
        # Número de iteraciones de optimización llevadas a cabo
        self.iter_optimizacion = None
        # Mejor individuo de la población
        self.mejor_individuo = None
        # Fitness del mejor individuo de la población (el de mayor fitness)
        self.mejor_fitness = None
        # Valor de la métrica del mejor individuo de la población
        self.mejor_valor_metrica = None
        # Secuencia del mejor individuo de la población
        self.mejor_secuencia = None
        # Índice de las columnas que incluye como predictores el mejor individuo
        # de la población
        self.mejor_predictores = None
        # Información de todas los individuos de la población en cada generación
        self.historico_individuos = []
        # Secuencia del mejor individuo en cada generación
        self.historico_mejor_secuencia = []
        # Índice de las columnas que incluye como predictores el mejor individuo
        # en cada generación.
        self.historico_mejor_predictores = []
        # Fitness del mejor individuo en cada generación
        self.historico_mejor_fitness = []
        # Valor de la métrica del mejor individuo en cada generación
        self.historico_mejor_valor_metrica = []
        # Diferencia absoluta entre el mejor fitness de generaciones consecutivas
        self.diferencia_abs = []
        # data.frame con la información del mejor fitness y valor de variables
        # encontrado en cada generación, así como la diferencia respecto a la 
        # generación anterior.
        self.resultados_df = None
        # Fitness del mejor individuo de todas las generaciones
        self.fitness_optimo = None
        # Secuencia del mejor individuo de todas las generaciones
        self.secuencia_optima = None
        # Índice de las columnas incluidas como predictores en el mejor individuo
        # de todas las generaciones.
        self.predictores_optimos = None
        # Valor de función objetivo del mejor individuo de todas las generaciones
        self.valor_metrica_optimo = None

        # COMPROBACIONES INICIALES: EXCEPTIONS Y WARNINGS
        # ----------------------------------------------------------------------
        if self.n_max is not None and self.n_max > self.n_variables:
            raise Exception(
                "El valor de n_max no puede ser superior al de n_variables"
            )

        # COMPROBACIONES INICIALES: ACCIONES
        # ----------------------------------------------------------------------
        # Si no se especifica n_max, se emplea por defecto el valor de 
        # n_variables.
        if self.n_max is None:
            self.n_max = n_variables

        # SE CREAN LOS INDIVIDUOS DE LA POBLACIÓN Y SE ALMACENAN
        # ----------------------------------------------------------------------
        for i in np.arange(n_individuos):
            individuo_i = Individuo(
                            n_variables = self.n_variables,
                            n_max = self.n_max,
                            n_min = self.n_min,
                            verbose = verbose
                          )
            self.individuos.append(individuo_i)

        # INFORMACIÓN DEL PROCESO (VERBOSE)
        # ----------------------------------------------------------------------
        if verbose:
            print("----------------")
            print("Población creada")
            print("----------------")
            print("Número de individuos: " + str(self.n_individuos))
            print("Número máximo de predictores iniciales: " + str(self.n_max))
            print("Número mínimo de predictores iniciales: " + str(self.n_min))
            print("")

    def __repr__(self):

        texto = "============================" \
                + "\n" \
                + "         Población" \
                + "\n" \
                + "============================" \
                + "\n" \
                + "Número de individuos: " + str(self.n_individuos) \
                + "\n" \
                + "Número máximo de predictores iniciales: " + str(self.n_max) \
                + "\n" \
                + "Número mínimo de predictores iniciales: " + str(self.n_min) \
                + "\n" \
                + "Evaluada: " + str(self.evaluada) \
                + "\n" \
                + "Optimizada: " + str(self.optimizada) \
                + "\n" \
                + "Métrica de evaluación: " + str(self.metrica) \
                + "\n" \
                + "Modelo: " + str(self.modelo) \
                + "\n" \
                + "Iteraciones optimización (generaciones): " \
                    + str(self.iter_optimizacion) \
                + "\n" \
                + "\n" \
                + "Información del mejor individuo:" \
                + "\n" \
                + "--------------------------------" \
                + "\n" \
                + "Secuencia: " + str(self.mejor_secuencia) \
                + "\n" \
                + "Índice predictores: " + str(self.mejor_predictores) \
                + "\n" \
                + "Fitness: " + str(self.mejor_fitness) \
                + "\n" \
                + "\n" \
                + "Resultados tras optimizar:" \
                + "\n" \
                + "--------------------------" \
                + "\n" \
                + "Secuencia óptima: " + str(self.secuencia_optima) \
                + "\n" \
                + "Índice predictores óptimos: " + str(self.predictores_optimos) \
                + "\n" \
                + "Valor óptimo métrica: " + str(self.valor_metrica_optimo) \
                + "\n" \
                + "Fitness óptimo: " + str(self.fitness_optimo)
                
        return(texto)

    def mostrar_individuos(self, n=None):

        if n is None:
            n = self.n_individuos
        elif n > self.n_individuos:
            n = self.n_individuos

        for i in np.arange(n):
            print(self.individuos[i])
            
        return(None)


    def evaluar_poblacion(self, x, y, tipo_modelo, modelo, metrica, cv=5,
                          test_size=0.2, cv_seed=123, forzar_evaluacion = True,
                          rf_n_estimators=100, nivel_referencia = None, verbose = False):

        # COMPROBACIONES INICIALES: EXCEPTIONS Y WARNINGS
        # ----------------------------------------------------------------------
        if metrica not in ["neg_mean_squared_error",
                           "neg_mean_absolute_error",
                           "f1", "accuracy"]:
            raise Exception(
                "La métrica debe ser: neg_mean_squared_error, " \
                 + "neg_mean_absolute_error, f1 o accuracy."
            )

        if self.n_variables != x.shape[1]:
            raise Exception(
                "n_variables debe ser igual al número de columnas de x."
            )

        if not isinstance(x, np.ndarray) or x.ndim != 2:
            raise Exception(
                "x debe ser un array numpy de dos dimensiones (una matriz)."
            )
        if not isinstance(y, np.ndarray) or y.ndim != 1:
            raise Exception(
                "y debe ser un array numpy de 1 dimensiones."
            )

        if modelo == "lineal":
            if metrica not in ["neg_mean_squared_error", "neg_mean_absolute_error"]:
                raise Exception(
                "Para el modelo lineal, la métrica debe ser: " \
                + "neg_mean_squared_error o neg_mean_absolute_error."
                )
            if tipo_modelo != "regresion":
                raise Exception(
                "El modelo lineal solo puede aplicarse a problemas de regresión."
                )

        if modelo == "glm":
            if metrica not in ["f1", "accuracy"]:
                raise Exception(
                "Para el modelo glm la métrica de evaluación debe ser f1," \
                + "o accuracy."
                )
            if tipo_modelo != "clasificacion":
                raise Exception(
                "El modelo glm solo puede aplicarse a problemas de clasificación."
                )
            if len(np.unique(y)) != 1:
                raise Exception(
                "El modelo glm solo puede aplicarse a problemas de clasificación" \
                + "binaria."
                )

        # SE EVALÚA CADA INDIVIDUO DE LA POBLACIÓN
        # ----------------------------------------------------------------------
        self.metrica = metrica
        self.modelo  = modelo

        for i in np.arange(self.n_individuos):
            if forzar_evaluacion:
                #Se evaluan todos los individuos
                self.individuos[i].evaluar_individuo(
                    x = x,
                    y = y,
                    cv = cv,
                    test_size = test_size,
                    tipo_modelo = tipo_modelo,
                    modelo = modelo,
                    metrica = metrica,
                    rf_n_estimators = rf_n_estimators,
                    verbose = verbose
                )
            else:
                if self.individuos[i].fitness is None:
                    # Solo los no previamente evaluados se evaluan
                    self.individuos[i].evaluar_individuo(
                        x = x,
                        y = y,
                        cv = cv,
                        test_size = test_size,
                        tipo_modelo = tipo_modelo,
                        modelo = modelo,
                        metrica = metrica,
                        rf_n_estimators = rf_n_estimators,
                        verbose = verbose
                    )

        # MEJOR INDIVIDUO DE LA POBLACIÓN
        # ----------------------------------------------------------------------
        # Se identifica el mejor individuo de toda el población, el de mayor
        # fitness.

        # Se selecciona inicialmente como mejor individuo el primero.
        self.mejor_individuo = copy.deepcopy(self.individuos[0])
        # Se comparan todas los individuos de la población.
        for i in np.arange(1,self.n_individuos):
            if self.individuos[i].fitness > self.mejor_individuo.fitness:
                self.mejor_individuo = copy.deepcopy(self.individuos[i])

        # Se extrae la información del mejor individuo de la población.
        self.mejor_fitness = copy.copy(self.mejor_individuo.fitness)
        self.mejor_valor_metrica = copy.copy(self.mejor_individuo.valor_metrica)
        self.mejor_secuencia = copy.copy(self.mejor_individuo.secuencia)
        self.mejor_predictores = copy.copy(self.mejor_individuo.predictores)
        
        self.evaluada = True
        
        # INFORMACIÓN DEL PROCESO (VERBOSE)
        # ----------------------------------------------------------------------
        if verbose:
            print("------------------")
            print("Población evaluada")
            print("------------------")
            print("Mejor fitness encontrado : " + str(self.mejor_fitness))
            print("Mejor valor de la métrica: " + str(self.mejor_valor_metrica))
            print("Mejor secuencia encontrada: " 
                + str(self.mejor_secuencia))
            print("Mejores predictores encontrados: " 
                + str(self.mejor_predictores))
            print("")


    def cruzar_individuos(self, parental_1, parental_2, metodo_cruce = "uniforme",
                          verbose=False):

        # COMPROBACIONES INICIALES: EXCEPTIONS Y WARNINGS
        # ----------------------------------------------------------------------
        if parental_1 not in np.arange(self.n_individuos):
            raise Exception(
                "El índice del parental_1 debe de ser un valor entre 0 y " +
                "el número de individuos de la población."
                )
        if parental_2 not in np.arange(self.n_individuos):
            raise Exception(
                "El índice del parental_2 debe de ser un valor entre 0 y " +
                "el número de individuos de la población."
                )

        if metodo_cruce not in ["uniforme", "punto_simple"]:
            raise Exception(
                "El argumento metodo_cruce debe de ser" +
                "uniforme o punto_simple."
                )

        # CREACIÓN DE LA DESCENDENCIA
        # ----------------------------------------------------------------------
        # Se extraen los parentales acorde a los índices indicados.
        parental_1 = self.individuos[parental_1]
        parental_2 = self.individuos[parental_2]
        
        # Se clona uno de los parentales para utilizarlo como plantilla del nuevo
        # individuo.
        descendencia = copy.deepcopy(parental_1)
        descendencia.secuencia = np.repeat(None, descendencia.n_variables)
        descendencia.predictores = None
        descendencia.fitness = None
        descendencia.valor_metrica = None
        descendencia.n_predictores_incluidos = None

        if metodo_cruce == "uniforme":
            # Se seleccionan aleatoriamente las posiciones que se heredan del
            # parental_1 y del parental 2.
            herencia_parent_1 = np.random.choice(
                                    a       = [True, False],
                                    size    = descendencia.n_variables,
                                    p       = [0.5, 0.5],
                                    replace = True
                                )
            herencia_parent_2 = np.logical_not(herencia_parent_1)

            # Se transfieren los valores al nuevo individuo.
            descendencia.secuencia[herencia_parent_1] \
                = parental_1.secuencia[herencia_parent_1]

            descendencia.secuencia[herencia_parent_2] \
                = parental_2.secuencia[herencia_parent_2]
            
        if metodo_cruce == "punto_simple":
            punto_cruce  = np.random.choice(
                            a = np.arange(1, descendencia.n_variables - 1),
                            size = 1
                            )
            punto_cruce = punto_cruce[0]
            descendencia.secuencia = np.hstack(   
                                        (parental_1.secuencia[:punto_cruce],
                                        parental_2.secuencia[punto_cruce:])
                                    )
            
        # Todo individuo debe tener como mínimo 1 predictor, si como consecuencia del 
        # cruzamiento, ningun valor de la secuencia es True, se selecciona una posición
        # aleatoria y se sobreescribe con True.
        if sum(descendencia.secuencia == True) == 0:
            indice = np.random.choice(
                        a       = np.arange(descendencia.n_variables),
                        size    = 1, 
                        replace = False
                      )
            descendencia.secuencia[indice] = True

        descendencia.secuencia = descendencia.secuencia.astype('bool')
        descendencia.predictores \
            = np.arange(descendencia.n_variables)[descendencia.secuencia]

        descendencia.n_predictores_incluidos = np.sum(descendencia.secuencia)
        # Se crea un deepcopy para que el nuevo individuo sea independiente de 
        # los parentales. Esto evita problemas si posteriormente se muta.
        descendencia = copy.deepcopy(descendencia)
            
        

        # INFORMACIÓN DEL PROCESO (VERBOSE)
        # ----------------------------------------------------------------------
        if verbose:
            print("---------------")
            print("Cruce realizado")
            print("---------------")
            print("Secuencia: " + str(descendencia.secuencia))
            print("Índice predictores: " + str(descendencia.predictores))
            print("")

        return(descendencia)


    def seleccionar_individuo(self, n, return_indices=True,
                              metodo_seleccion="tournament", verbose=False):


        # COMPROBACIONES INICIALES: EXCEPTIONS Y WARNINGS
        # ----------------------------------------------------------------------
        if metodo_seleccion not in ["ruleta", "rank", "tournament"]:
            raise Exception(
                "El método de selección debe de ser ruleta, rank o tournament."
                )
        
        # SELECCIÓN DE INDIVIDUOS
        # ----------------------------------------------------------------------
        # Se crea un array con el fitness de cada individuo de la población.
        array_fitness = np.full(self.n_individuos, None, dtype = "float")
        for i in np.arange(self.n_individuos):
            array_fitness[i] = copy.copy(self.individuos[i].fitness)
        
        # Se calcula la probabilidad de selección de cada individuo en función
        # de su fitness.
        if metodo_seleccion == "ruleta":
            if self.metrica in ["neg_mean_squared_error", "neg_mean_absolute_error"]:
                # Si el fitness es [-inf,0] se emplea 1/fitness
                array_fitness = 1/array_fitness

            probabilidad_seleccion = array_fitness / np.sum(array_fitness)
            ind_seleccionado = np.random.choice(
                                    a       = np.arange(self.n_individuos),
                                    size    = n,
                                    p       = list(probabilidad_seleccion),
                                    replace = True
                               )
        elif metodo_seleccion == "rank":
            # La probabilidad con este método es inversamente proporcional a la
            # posición en la que quedan ordenados los individuos de menor a mayor
            # fitness.
            ranks = rankdata(-1*array_fitness)
            probabilidad_seleccion = 1 / ranks
            probabilidad_seleccion = probabilidad_seleccion / np.sum(probabilidad_seleccion)
            ind_seleccionado = np.random.choice(
                                a       = np.arange(self.n_individuos),
                                size    = n,
                                p       = list(probabilidad_seleccion),
                                replace = True
                            )
        elif metodo_seleccion == "tournament":
            if self.metrica in ["neg_mean_squared_error", "neg_mean_absolute_error"]:
                # Si el fitness es [-inf,0] se emplea 1/fitness
                array_fitness = 1/array_fitness

            ind_seleccionado = np.repeat(None,n)
            for i in np.arange(n):
                # Se seleccionan aleatoriamente dos parejas de individuos.
                candidatos_a = np.random.choice(
                                a       = np.arange(self.n_individuos),
                                size    = 2,
                                replace = False
                            )
                candidatos_b = np.random.choice(
                                a       = np.arange(self.n_individuos),
                                size    = 2,
                                replace = False
                            )
                # De cada pareja se selecciona el de mayor fitness.
                if array_fitness[candidatos_a[0]] > array_fitness[candidatos_a[1]]:
                    ganador_a = candidatos_a[0]
                else:
                    ganador_a = candidatos_a[1]

                if array_fitness[candidatos_b[0]] > array_fitness[candidatos_b[1]]:
                    ganador_b = candidatos_b[0]
                else:
                    ganador_b = candidatos_b[1]

                # Se comparan los dos ganadores de cada pareja.
                if array_fitness[ganador_a] > array_fitness[ganador_b]:
                    ind_final = ganador_a
                else:
                    ind_final = ganador_b
                
                ind_seleccionado[i] = ind_final

        # INFORMACIÓN DEL PROCESO (VERBOSE)
        # ----------------------------------------------------------------------
        if verbose:
            print("----------------------")
            print("Individuo seleccionado")
            print("----------------------")
            print("Método selección: " + metodo_seleccion)
            print("Índice seleccionado: " + str(ind_seleccionado))
            print("")

        if(return_indices):
            return(ind_seleccionado)
        else:
            if n == 1:
                return(copy.deepcopy(self.individuos[int(ind_seleccionado)]))
            if n > 1:
                return(
                    [copy.deepcopy(self.individuos[i]) for i in ind_seleccionado]
                )

    def crear_nueva_generacion(self, metodo_seleccion="tournament",
                               metodo_cruce = "uniforme",
                               elitismo=0.1, prob_mut=0.1,
                               verbose=False, verbose_seleccion=False,
                               verbose_cruce=False, verbose_mutacion=False):

        # Lista donde almacenar los individuos de la nueva generación.
        nuevos_individuos = []

        # ELITISMO
        # ----------------------------------------------------------------------
        if elitismo > 0:
            # Número de individuos que pasan directamente a la siguiente
            # generación.
            n_elitismo = int(np.ceil(self.n_individuos*elitismo))

            # Se identifican los n_elitismo individuos con mayor fitness (élite).
            array_fitness = np.zeros(shape = self.n_individuos, dtype=float)
            for i in np.arange(self.n_individuos):
                array_fitness[i] = copy.copy(self.individuos[i].fitness)
            rank = np.flip(np.argsort(array_fitness), axis = 0)
            elite = [copy.deepcopy(self.individuos[i]) for i in rank[:n_elitismo]]
            # Se añaden los individuos élite a la lista de nuevos individuos.
            nuevos_individuos = nuevos_individuos + elite
        else:
            n_elitismo = 0
            
        # CREACIÓN DE NUEVOS INDIVIDUOS POR CRUCES
        # ----------------------------------------------------------------------
        for i in np.arange(self.n_individuos-n_elitismo):
            # Seleccionar parentales
            indice_parentales = self.seleccionar_individuo(
                                    n                = 2,
                                    return_indices   = True,
                                    metodo_seleccion = metodo_seleccion,
                                    verbose          = verbose_seleccion
                                 )
            # Cruzar parentales para obtener la descendencia
            descendencia = self.cruzar_individuos(
                            parental_1   = indice_parentales[0],
                            parental_2   = indice_parentales[1],
                            metodo_cruce = metodo_cruce,
                            verbose      = verbose_cruce
                           )
            # Mutar la descendencia
            descendencia.mutar(
                prob_mut         = prob_mut,
                verbose          = verbose_mutacion
            )

            # Si n_max_estricto=True, se elimina el exceso de Trues en la
            # secuencia de la descendencia.
            if self.n_max_estricto:
                descendencia.forzar_n_max()

            # Se añade la descendencia a la lista de nuevos individuos. Para
            # que no de error la unión, se introduce el individuo descendencia
            # dentro de una lista.
            nuevos_individuos.append(copy.deepcopy(descendencia))

        # ACTUALIZACIÓN INFORMACIÓN DE LA POBLACIÓN
        # ----------------------------------------------------------------------
        self.individuos = copy.deepcopy(nuevos_individuos)
        self.mejor_individuo = None
        self.mejor_fitness = None
        self.mejor_valor_metrica = None
        self.mejor_secuencia = None
        self.mejor_predictores = None
        self.evaluada = False
        
        # INFORMACIÓN DEL PROCESO (VERBOSE)
        # ----------------------------------------------------------------------
        if verbose:
            print("----------------------")
            print("Nueva población creada")
            print("----------------------")
            print("Método selección: " + metodo_seleccion)
            print("Elitismo: " + str(elitismo))
            print("Número individuos élite: " + str(n_elitismo))
            print("Número de nuevos individuos: "\
                + str(self.n_individuos-n_elitismo))
            print("")

    def optimizar(self, x, y, tipo_modelo, modelo, metrica, cv = 5,
                  test_size=0.2, cv_seed=123, nivel_referencia = None,
                  n_generaciones = 50, metodo_seleccion="tournament",
                  metodo_cruce = "uniforme", elitismo=0.1,
                  prob_mut=0.1, rf_n_estimators=100,
                  parada_temprana=False, rondas_parada=None,
                  tolerancia_parada=None,verbose=False,
                  verbose_nueva_generacion=False,
                  verbose_seleccion=False, verbose_cruce=False,
                  verbose_mutacion=False, verbose_evaluacion=False):

        # COMPROBACIONES INICIALES: EXCEPTIONS Y WARNINGS
        # ----------------------------------------------------------------------
        # Si se activa la parada temprana, hay que especificar los argumentos
        # rondas_parada y tolerancia_parada.
        if parada_temprana \
        and (rondas_parada is None or tolerancia_parada is None):
            raise Exception(
                "Para activar la parada temprana es necesario indicar un " \
                + " valor de rondas_parada y de tolerancia_parada."
                )

        # ITERACIONES (GENERACIONES)
        # ----------------------------------------------------------------------
        start = time.time()

        for i in np.arange(n_generaciones):
            if verbose:
                print("-------------")
                print("Generación: " + str(i))
                print("-------------")

            # En la primera iteración, la población ya está creada
            if i > 0:
                # CREAR UNA NUEVA GENERACIÓN
                # --------------------------------------------------------------    
                self.crear_nueva_generacion(
                    metodo_seleccion   = metodo_seleccion,
                    metodo_cruce       = metodo_cruce,
                    elitismo           = elitismo,
                    prob_mut           = prob_mut,
                    verbose            = verbose_nueva_generacion,
                    verbose_seleccion  = verbose_seleccion,
                    verbose_cruce      = verbose_cruce,
                    verbose_mutacion   = verbose_mutacion
                    )
            
            # EVALUAR INDIVIDUOS DE LA POBLACIÓN
            # ------------------------------------------------------------------
            self.evaluar_poblacion(
                x  = x,
                y  = y,
                cv = cv,
                test_size   = test_size,
                cv_seed     = cv_seed,
                tipo_modelo = tipo_modelo,
                modelo      = modelo,
                metrica     = metrica,
                forzar_evaluacion = False,
                rf_n_estimators = rf_n_estimators,
                verbose     = verbose_evaluacion
                )

            # SE ALMACENA LA INFORMACIÓN DE LA GENERACIÓN EN LOS HISTÓRICOS
            # ------------------------------------------------------------------
            self.historico_individuos.append(copy.deepcopy(self.individuos))
            self.historico_mejor_fitness.append(copy.deepcopy(self.mejor_fitness))
            self.historico_mejor_secuencia.append(
                                    copy.deepcopy(self.mejor_secuencia)
                                )
            self.historico_mejor_predictores.append(
                                    copy.deepcopy(self.mejor_predictores)
                                )
            self.historico_mejor_valor_metrica.append(
                                    copy.deepcopy(self.mejor_valor_metrica)
                                )

            # SE CALCULA LA DIFERENCIA ABSOLUTA RESPECTO A LA GENERACIÓN ANTERIOR
            # ------------------------------------------------------------------
            # La diferencia solo puede calcularse a partir de la segunda
            # generación.
            if i == 0:
                self.diferencia_abs.append(None)
            else:
                diferencia = abs(self.historico_mejor_fitness[i] \
                                 - self.historico_mejor_fitness[i-1])
                self.diferencia_abs.append(diferencia)

            # CRITERIO DE PARADA
            # ------------------------------------------------------------------
            # Si durante las últimas n generaciones, la diferencia absoluta entre
            # mejores individuos no es superior al valor de tolerancia_parada,
            # se detiene el algoritmo y no se crean nuevas generaciones.
            if parada_temprana and i > rondas_parada:
                ultimos_n = np.array(self.diferencia_abs[-(rondas_parada): ])
                if all(ultimos_n < tolerancia_parada):
                    print("Algoritmo detenido en la generación " 
                          + str(i) \
                          + " por falta cambio absoluto mínimo de " \
                          + str(tolerancia_parada) \
                          + " durante " \
                          + str(rondas_parada) \
                          + " generaciones consecutivas.")
                    break

        end = time.time()
        self.optimizada = True
        self.iter_optimizacion = i + 1
        
        # IDENTIFICACIÓN DEL MEJOR INDIVIDUO DE TODO EL PROCESO
        # ----------------------------------------------------------------------
        indice_valor_optimo  = np.argmax(np.array(self.historico_mejor_fitness))
        self.fitness_optimo  = self.historico_mejor_fitness[indice_valor_optimo]
        self.valor_metrica_optimo= self \
                             .historico_mejor_valor_metrica[indice_valor_optimo]
        self.secuencia_optima = self \
                                .historico_mejor_secuencia[indice_valor_optimo]
        self.predictores_optimos = self \
                                .historico_mejor_predictores[indice_valor_optimo]
        
        # CREACIÓN DE UN DATAFRAME CON LOS RESULTADOS
        # ----------------------------------------------------------------------
        self.resultados_df = pd.DataFrame(
            {
            "mejor_fitness"        : self.historico_mejor_fitness,
            "mejor_valor_metrica"  : self.historico_mejor_valor_metrica,
            "mejor_secuencia"      : self.historico_mejor_secuencia,
            "mejor_predictores"    : self.historico_mejor_predictores,
            "diferencia_abs"       : self.diferencia_abs
            }
        )
        self.resultados_df["generacion"] = self.resultados_df.index
        
        print("-------------------------------------------")
        print("Optimización finalizada " \
              + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        print("-------------------------------------------")
        print("Duración optimización: " + str(end - start))
        print("Número de generaciones: " + str(self.iter_optimizacion))
        print("Secuencia óptima: " + str(self.secuencia_optima))
        print("Predictores óptimos: " + str(self.predictores_optimos))
        print("Valor métrica óptimo: " + str(self.valor_metrica_optimo))
        print("")

    def plot_evolucion_fitness(self):

        if not self.optimizada:
            raise Exception(
                "El gráfico solo puede generarse si la población ha sido " \
                    + "optimizada previamente."
            )
        plt.style.use('ggplot')
        fig, ax = plt.subplots()
        self.resultados_df.plot(
            x = "generacion",
            y = "mejor_fitness",
            ax = ax
        )
        ax.set(title='Evolución del mejor Individuo',
               xlabel='generacion', ylabel='fitness')
        ax.legend().set_visible(False)

    def plot_frecuencia_seleccion(self):

        if not self.optimizada:
            raise Exception(
                "El gráfico solo puede generarse si la población ha sido " \
                    + "optimizada previamente."
            )
        unique, counts =  np.unique(
                            np.concatenate(
                                self.resultados_df.mejor_predictores.ravel(),
                                axis = 0
                            ),
                            return_counts=True
                          )
        frecuencia = 100* (counts/self.iter_optimizacion)
        frecuencia_selecion = pd.DataFrame(
                                {"predictor":unique,
                                 "frecuencia" : frecuencia}) \
                              .sort_values(
                                    by=["frecuencia"],
                                    ascending = False
                               )

        plt.style.use('ggplot')
        fig, ax = plt.subplots()
        frecuencia_selecion.plot.barh(
            x = "predictor",
            y = "frecuencia",
            ax = ax
        )
        ax.set(title='Frecuencia de selección',
               xlabel='frecuencia', ylabel='predictor')
        ax.legend().set_visible(False)

### MODELO DE CLASIFICACIÓN

#### PREPARACIÓN DE DATOS

In [4]:
import pandas as pd
import numpy as np

In [5]:
datos = pd.read_csv('TAREA1.csv', sep=',', header=1)

In [6]:
datos['day_of_week'].unique()

array(['mon', 'tue', 'wed', 'thu', 'fri'], dtype=object)

In [7]:
day_dict = {'mon': 1, 'tue':2, 'wed':3, 'thu':4, 'fri':5}
datos['day_of_week'].replace(day_dict, inplace= True)

In [8]:
datos['month'].unique()

array(['may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'mar', 'apr',
       'sep'], dtype=object)

In [9]:
month_dict ={'may':5, 'jun':6, 'jul':7, 'aug':8, 'oct':10, 'nov':11, 'dec':12, 'mar':3, 'apr':4,
       'sep':9}
datos['month'].replace(month_dict, inplace=True)

In [10]:
datos['poutcome'].unique()

array(['nonexistent', 'failure', 'success'], dtype=object)

In [11]:
poutcome_dict = {'nonexistent': 0, 'failure': 1, 'success': 2}
datos['poutcome'].replace(poutcome_dict, inplace=True)

In [12]:
datos['job'].unique()

array(['housemaid', 'services', 'admin.', 'blue-collar', 'technician',
       'retired', 'management', 'unemployed', 'self-employed', 'unknown',
       'entrepreneur', 'student'], dtype=object)

In [13]:
job_dict = {'housemaid':1, 'services':2, 'admin.':3, 'blue-collar':4, 'technician':5,
       'retired':6, 'management':7, 'unemployed':8, 'self-employed':9, 'unknown':0,
       'entrepreneur':10, 'student':11}
datos['job'].replace(job_dict, inplace=True)

In [14]:
datos['marital'].unique()

array(['married', 'single', 'divorced', 'unknown'], dtype=object)

In [15]:
marital_dict ={'married':2, 'single':1, 'divorced':3, 'unknown':0}
datos['marital'].replace(marital_dict, inplace=True)

In [16]:
datos['education'].unique()

array(['basic.4y', 'high.school', 'basic.6y', 'basic.9y',
       'professional.course', 'unknown', 'university.degree',
       'illiterate'], dtype=object)

In [17]:
education_dict ={'basic.4y':2, 'high.school':5, 'basic.6y':3, 'basic.9y':4,
       'professional.course':6, 'unknown':0, 'university.degree':7,
       'illiterate':1}
datos['education'].replace(education_dict, inplace=True)

In [18]:
datos['default'].unique()

array(['no', 'unknown', 'yes'], dtype=object)

In [19]:
default_dict ={'no':2, 'unknown':0, 'yes':1}
datos['default'].replace(default_dict, inplace=True)

In [20]:
datos['housing'].unique()
datos['housing'].replace(default_dict, inplace=True)

In [21]:
datos['loan'].unique()
datos['loan'].replace(default_dict, inplace=True)

In [22]:
datos['contact'].unique()

array(['telephone', 'cellular'], dtype=object)

In [23]:
contact_dict ={'telephone':1, 'cellular':2}
datos['contact'].replace(contact_dict, inplace=True)

In [24]:
datos['y'].unique()

array(['no', 'yes'], dtype=object)

In [25]:
y_dict ={'yes':1, 'no':0}
datos['y'].replace(y_dict, inplace=True)

In [26]:
variab = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 
          'contact', 'month', 'day_of_week', 'duration','campaign', 'pdays', 'previous',
          'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

In [27]:
x = datos[variab].values
x = (x - np.mean(x)) / np.std(x)
y = datos['y'].values

### APLICACIÓN ALGORÍTMO - CLASIFICACIÓN

In [ ]:
poblacion = Poblacion(n_individuos   = 50,
                n_variables    = x.shape[1],
                n_max          = 10,
                n_max_estricto = False,    
                n_min          = 1,
                verbose        = False)

poblacion.optimizar(
    x                  = x,
    y                  = y,
    cv                 = 3,
    test_size          =  0.3,
    tipo_modelo        = "clasificacion",
    modelo             = "randomforest",
    rf_n_estimators    = 50,
    metrica            = "accuracy",
    n_generaciones     = 50,
    metodo_seleccion   = "ruleta",
    metodo_cruce       = "uniforme",
    elitismo           = 0.05,
    prob_mut           = 0.1,
    parada_temprana    = True,
    rondas_parada      = 5,
    tolerancia_parada  = 0.01,
    verbose            = True,
    verbose_nueva_generacion = False,
    verbose_seleccion        = False,
    verbose_cruce            = False,
    verbose_mutacion         = False,
    verbose_evaluacion       = False    
)

-------------
Generación: 0
-------------
-------------
Generación: 1
-------------
-------------
Generación: 2
-------------
-------------
Generación: 3
-------------
-------------
Generación: 4
-------------
-------------
Generación: 5
-------------
-------------
Generación: 6
-------------


In [ ]:
print(poblacion)

# Evolución de la optimización
poblacion.plot_evolucion_fitness()

# Frecuencia relativa selección predictores
poblacion.plot_frecuencia_seleccion()

### MODELO PREDICCIÓN CONTINUA

In [ ]:
datos = pd.read_csv('Portfolio.csv', sep=',', header=None)

In [ ]:
df = datos.drop([0], axis=1)
df

In [ ]:
%matplotlib inline
# Para que las imágenes se muestren en el centro de la celda.
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [ ]:
data = df.values
normdata = (data-np.mean(data))/np.std(data)


In [ ]:
x = normdata[:,1:]
y = normdata[:,0]


poblacion = Poblacion(n_individuos   = 10,
                n_variables    = x.shape[1],
                n_max          = 10,
                n_max_estricto = False,    
                n_min          = 1,
                verbose        = False)

poblacion.optimizar(
    x                  = x,
    y                  = y,
    cv                 = 3,
    test_size          =  0.3,
    tipo_modelo        = "regresion",
    modelo             = "randomforest",
    rf_n_estimators    = 50,
    metrica            = "neg_mean_squared_error",
    n_generaciones     = 50,
    metodo_seleccion   = "ruleta",
    metodo_cruce       = "uniforme",
    elitismo           = 0.05,
    prob_mut           = 0.1,
    parada_temprana    = True,
    rondas_parada      = 5,
    tolerancia_parada  = 0.01,
    verbose            = True,
    verbose_nueva_generacion = False,
    verbose_seleccion        = False,
    verbose_cruce            = False,
    verbose_mutacion         = False,
    verbose_evaluacion       = False    
)

In [ ]:
print(poblacion)

# Evolución de la optimización
poblacion.plot_evolucion_fitness()

# Frecuencia relativa selección predictores
poblacion.plot_frecuencia_seleccion()

In [ ]:
print (poblacion.mejor_individuo.__repr__())